# Part A: Speckle Filtering SAR data
## Lee Filter Function


In [1]:
!pip install geemap

import ee
import numpy as np
import geemap.eefolium as geemap
import pprint

ee.Authenticate()
ee.Initialize()

     |████████████████████████████████| 460kB 5.1MB/s 
     |████████████████████████████████| 225kB 9.2MB/s 
     |████████████████████████████████| 1.6MB 10.9MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 102kB 9.7MB/s 
     |████████████████████████████████| 143kB 33.5MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 1.3MB 31.5MB/s 
     |████████████████████████████████| 1.2MB 35.6MB/s 
     |████████████████████████████████| 5.1MB 40.6MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
     |████████████████████████████████| 552kB 37.0MB/s 
     |████████████████████████████████| 122kB 52.2MB/s 
     |████████████████████████████████| 389kB 41.8MB/s 
     |████████████████████████████████| 122kB 54.2MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 81kB 7.9MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
  Cre

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=NfI7ye1jnntPLXuNikNE7KGMyYuESlGcD50BIldSW24&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4IDem8ooqg19fHRfeoJV-ObC7CGXNy4vyzQjyPGy6Xqfw6wH2JcLo

Successfully saved authorization token.


Before we start, let's set up a few functions that we will use in the script

In [2]:
# Functions

#####
#Function to convert from dB
def toNatural(img):
    return ee.Image(10.0).pow(img.select(0).divide(10.0))

#Function to convert to dB
def toDB(img):
    return ee.Image(img).log10().multiply(10.0)

#the Lee filter is a commonly used filter for SAR imagery
# see:  https:#earth.esa.int/documents/653194/656796/Speckle_Filtering.pdf
# uses directional masks to determine the most homogeneous part of
# the sliding window where local statistics have to be estimated

########## Lee filter
def RefinedLee(img):
    # img must be in natural units, i.e. not in dB!
    # Set up 3x3 kernels

    # convert to natural.. do not apply function on dB!
    # our images are in dB so we need to apply the toNatural algorithm
    myimg = toNatural(img)

    weights3 = ee.List.repeat(ee.List.repeat(1,3),3)
    kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False)

    mean3 = myimg.reduceNeighborhood(ee.Reducer.mean(), kernel3)
    variance3 = myimg.reduceNeighborhood(ee.Reducer.variance(), kernel3)

    # Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
    sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0], [0,1,0,1,0,1,0], [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]])

    sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False)

    # Calculate mean and variance for the sampled windows and store as 9 bands
    sample_mean = mean3.neighborhoodToBands(sample_kernel)
    sample_var = variance3.neighborhoodToBands(sample_kernel)

    # Determine the 4 gradients for the sampled windows
    gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
    gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
    gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
    gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

    # And find the maximum gradient amongst gradient bands
    max_gradient = gradients.reduce(ee.Reducer.max())

    # Create a mask for band pixels that are the maximum gradient
    gradmask = gradients.eq(max_gradient)

    # duplicate gradmask bands: each gradient represents 2 directions
    gradmask = gradmask.addBands(gradmask)

    # Determine the 8 directions
    directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1)
    directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
    directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
    directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))
    # The next 4 are the not() of the previous 4
    directions = directions.addBands(directions.select(0).Not().multiply(5))
    directions = directions.addBands(directions.select(1).Not().multiply(6))
    directions = directions.addBands(directions.select(2).Not().multiply(7))
    directions = directions.addBands(directions.select(3).Not().multiply(8))

    # Mask all values that are not 1-8
    directions = directions.updateMask(gradmask)

    # "collapse" the stack into a singe band image (due to masking, each pixel has just one value (1-8) in it's directional band, and is otherwise masked)
    directions = directions.reduce(ee.Reducer.sum())

    sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))

    # Calculate localNoiseVariance
    sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0])

    # Set up the 7*7 kernels for directional statistics
    rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4))

    diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0], [1,1,1,0,0,0,0],
                            [1,1,1,1,0,0,0], [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]])

    rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False)
    diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False)

    # Create stacks for mean and variance using the original kernels. Mask with relevant direction.
    dir_mean = myimg.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
    dir_var = myimg.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

    dir_mean = dir_mean.addBands(myimg.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
    dir_var= dir_var.addBands(myimg.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

    # and add the bands for rotated kernels
    for i in range(1,4):
        dir_mean = dir_mean.addBands(myimg.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
        dir_var = dir_var.addBands(myimg.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
        dir_mean = dir_mean.addBands(myimg.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
        dir_var = dir_var.addBands(myimg.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))

    # "collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
    dir_mean = dir_mean.reduce(ee.Reducer.sum())
    dir_var= dir_var.reduce(ee.Reducer.sum())

    # A finally generate the filtered value
    varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))

    b = varX.divide(dir_var)

    result = dir_mean.add(b.multiply(myimg.subtract(dir_mean)))
    #return(result)
    return(img.addBands(ee.Image(toDB(result.arrayGet(0))).rename("filter")))
    #end of function

### Print functions

# Print out all the band ids for an image json
def printImageBands(image):
  print('Bands of example image:')
  for band in range(len(image['bands'])):
    print(image['bands'][band]['id'])
  print('\n')

# Print the size of a json image collection
def printCollectionSize(collection):
  print('Size of image collection:')
  print(len(collection['features']))
  print('\n')

# Get an image json object from a json image collection based on an index value
def getImageFromCollection(collection, index=0):
  return collection['features'][index]

# Print all image ids from a json image collection
def printImageNames(collection):
  print('All images in image collection:')
  for img in range(len(collection['features'])):
    print(str(img) + ': ' + collection['features'][img]['id'])
  print('\n')

# Part B: Visualize Sea Ice - Tuktoyuktuk

Let's import some Sentinel-1 data for Tuktoyuktuk
* Since we are looking at Sea Ice we will use Extra Wide mode, which has HH and HV bands

* We will map over this image collection and run the Lee Filter on each image 

In [3]:
aoi = ee.Geometry.Point(-130.3, 69.0)

# Import sentinel 1 and filter data series 
s1_stack =  ee.ImageCollection('COPERNICUS/S1_GRD') \
.filter(ee.Filter.eq('instrumentMode', 'EW')) \
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH')) \
.filterBounds(aoi)


collection = s1_stack.map(RefinedLee)


...and visualize the first image in the collection.

In [4]:
Map = geemap.Map(width = 1000, height = 1000)

S1_vis = {
  'bands': ['filter'],
  'min': -20,
  'max': 0,
  'gamma': 1.4,
}
Map.addLayer(collection.first(),  S1_vis,'S1-image')
Map.centerObject(aoi)
Map.addLayerControl()

Map


Let's also plot this as a multi-temporal RGB.

In [5]:
Map = geemap.Map(width = 1000, height = 1000)

# For each time period, create a median composite
VV1 = ee.Image(collection.filterDate('2017-09-01', '2017-11-30').median())
VV2 = ee.Image(collection.filterDate('2017-12-01', '2018-03-31').median())
VV3 = ee.Image(collection.filterDate('2018-04-01', '2018-06-30').median())

# Create a 3 band stack (one image for each time period)
MultiTemporalImage = VV1.addBands(VV2).addBands(VV3)  # This is an ee.Image (not imageCollection!)

# Get information about the bands as a list
band_names = MultiTemporalImage.bandNames()
print('Band names:', band_names.getInfo())  # ee.List of band names
# Note that we have multiple HH and HV bands now, and they are stored with _1 and _2

S1_vis = {
  'bands': ['HH', 'HH_1', 'HH_2'],
  'min': -20,
  'max': 0,
}


Map.addLayer(MultiTemporalImage,  S1_vis,'MultiTemporalImage')
Map.centerObject(aoi)
Map.addLayerControl()

Map

Band names: ['HH', 'HV', 'angle', 'filter', 'HH_1', 'HV_1', 'angle_1', 'filter_1', 'HH_2', 'HV_2', 'angle_2', 'filter_2']




---




# Part C: Visualize Kaskawulsh Glacier in ascending and descending mode

In [6]:
Map = geemap.Map(width=700,height=700)

# Kaskawulsh glacier
aoi = ee.Geometry.Point(-138.5, 60.8)

# Import sentinel 1 and filter data series 
s1_stack_asc =  ee.ImageCollection('COPERNICUS/S1_GRD') \
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
.filter(ee.Filter.eq('instrumentMode', 'IW')) \
.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) \
.filterBounds(aoi) \
.filterDate('2018-04-01','2018-07-31') 


s1_stack_desc =  ee.ImageCollection('COPERNICUS/S1_GRD') \
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
.filter(ee.Filter.eq('instrumentMode', 'IW')) \
.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
.filterBounds(aoi) \
.filterDate('2018-04-01','2018-07-31') 


collection_asc = s1_stack_asc.map(RefinedLee)
collection_desc = s1_stack_desc.map(RefinedLee)

S1_vis = {
  'bands': ['filter'],
  'min': -20,
  'max': 0,
  'gamma': 1.4,
}

Map.addLayer(collection_asc.first(), S1_vis,  "ASCENDING")
Map.addLayer(collection_desc.first(), S1_vis,  "DESCENDING")
Map.addLayerControl()
Map.centerObject(aoi,10)
Map

Let's visualize Single Date SAR in RGB (multi-band).


In [ ]:
Map = geemap.Map(width=700,height=700)

# In addition to Kaskawush (above) and Squamish (below), here are a couple of fun places to look at
# Peace Athabasca Delta
aoi = ee.Geometry.Point(-112, 58.5)

#Salt Plains Wood Buffalo
#aoi = ee.Geometry.Point(-112.2, 59.9)

#Bay of Fundy
#aoi = ee.Geometry.Point(-64.590, 45.696)

# import sentinel 1 and filter data series 
s1_stack_asc =  ee.ImageCollection('COPERNICUS/S1_GRD') \
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
.filter(ee.Filter.eq('instrumentMode', 'IW')) \
.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) \
.filterBounds(aoi) \
.filterDate('2018-08-01','2018-08-31') 

s1_stack_desc =  ee.ImageCollection('COPERNICUS/S1_GRD') \
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
.filter(ee.Filter.eq('instrumentMode', 'IW')) \
.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
.filterBounds(aoi) \
.filterDate('2018-08-01','2018-08-31') 


collection_asc = s1_stack_asc.map(RefinedLee)
collection_desc = s1_stack_desc.map(RefinedLee)

# Make an RGB color composite image (VV,VH,VV/VH).
rgb_asc = ee.Image.rgb(collection_asc.first().select('VV'),
                   collection_asc.first().select('VH'),
                   collection_asc.first().select('VV').divide(collection_asc.first().select('VH')))

rgb_desc = ee.Image.rgb(collection_desc.first().select('VV'),
                   collection_desc.first().select('VH'),
                   collection_desc.first().select('VV').divide(collection_desc.first().select('VH')))

# Add in google satellite for interpretation
url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Satellite', attribution='Google')

# Add the S1 rgb composite to the map object
Map.addLayer(rgb_asc, {'min': [-15, -25, 0], 'max': [-5, -5, 2]}, 'S1 Ascending RGB')
Map.addLayer(rgb_desc, {'min': [-15, -25, 0], 'max': [-5, -5, 2]}, 'S1 Descending RGB')


Map.addLayerControl()
Map.centerObject(aoi,10)

Map




---



# Part D: Identifying Shadow and Layover

The function below produces masks of areas of shadow and layover in SAR imagery.  Sentinel-1 imagery contains an "angle" band which is the viewing angle, not the local incidence angle. To compute the local incidence angle, a DEM is required. The script below (adapted from an implementation by Andreas Vollrath (ESA)) uses the SRTM DEM for this.  



In [34]:
import math

# This is a function that identifies area of layover and shadow, and performs terrain correction based on the SRTM DEM
def terrainCorrection(image):
  imgGeom = ee.Geometry(image.geometry())
  srtm = ee.Image('USGS/SRTMGL1_003').clip(imgGeom) # 30m srtm
  sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

  #  Radar geometry
  theta_i = image.select('angle')
  phi_i = ee.Terrain.aspect(theta_i) \
    .reduceRegion(ee.Reducer.mean(), theta_i.get('system:footprint'), 1000) \
    .get('aspect')

  #  Terrain geometry
  alpha_s = ee.Terrain.slope(srtm).select('slope')
  phi_s = ee.Terrain.aspect(srtm).select('aspect')

  #  Model geometry
  # Reduce to 3 angle
  phi_r = ee.Image.constant(phi_i).subtract(phi_s)

  # Ronvert all to radians
  phi_rRad = phi_r.multiply(math.pi / 180)
  alpha_sRad = alpha_s.multiply(math.pi / 180)
  theta_iRad = theta_i.multiply(math.pi / 180)
  ninetyRad = ee.Image.constant(90).multiply(math.pi / 180)

  # Slope steepness in range (eq. 2)
  alpha_r = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

  # Slope steepness in azimuth (eq 3)
  alpha_az = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

  # Local incidence angle (eq. 4)
  theta_lia = (alpha_az.cos().multiply((theta_iRad.subtract(alpha_r)).cos())).acos()
  theta_liaDeg = theta_lia.multiply(180 / math.pi)
 
  # Gamma_nought_flat
  gamma0 = sigma0Pow.divide(theta_iRad.cos())
  gamma0dB = ee.Image.constant(10).multiply(gamma0.log10())
  ratio_1 = gamma0dB.select('VV').subtract(gamma0dB.select('VH'))

  # Volumetric Model
  nominator = (ninetyRad.subtract(theta_iRad).add(alpha_r)).tan()
  denominator = (ninetyRad.subtract(theta_iRad)).tan()
  volModel = (nominator.divide(denominator)).abs()

  # Apply model
  gamma0_Volume = gamma0.divide(volModel)
  gamma0_VolumeDB = ee.Image.constant(10).multiply(gamma0_Volume.log10())

  # We add a layover/shadow mask to the original implmentation
  # layover, where slope > radar viewing angle
  alpha_rDeg = alpha_r.multiply(180 / math.pi)
  layover = alpha_rDeg.lt(theta_i)

  # Shadow where LIA > 85
  shadow = theta_liaDeg.lt(85)

  # Calculate the ratio for RGB vis
  ratio = gamma0_VolumeDB.select('VV').subtract(gamma0_VolumeDB.select('VH'))

  output = gamma0_VolumeDB.addBands(ratio).addBands(alpha_r).addBands(phi_s).addBands(theta_iRad) \
    .addBands(layover).addBands(shadow).addBands(gamma0dB).addBands(ratio_1)
  
  return image.addBands(output.select(['VV', 'VH', 'slope_1', 'slope_2'], ['VV', 'VH', 'layover', 'shadow']), None, True)

Let's implement this in Squamish BC.

In [36]:
Map = geemap.Map()
squamish = ee.Geometry.Point([-123.13535780567234, 49.694704992597494])

# import sentinel 1 and filter by mode, polarization date and bounds
s1_stack =  ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filterBounds(squamish) \
    .filterDate('2018-04-01','2018-07-31')


# make a subset of just the ascending mode images
s1_stack_asc =  s1_stack \
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    
# make a subset of just the descending mode images
s1_stack_desc =  s1_stack \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) 

corrected_asc = s1_stack_asc.map(terrainCorrection)
corrected_desc = s1_stack_desc.map(terrainCorrection)

S1_vis = {
  'bands': ['VV'],
  'min': -20,
  'max': 0,
  'gamma': 1.4,
}

S1_layover = {
    'bands': ['layover'],
}

S1_shadow = {
    'bands': ['shadow'],
}


Map.addLayer(s1_stack_asc.first(), S1_vis, "original ascending")
Map.addLayer(corrected_asc.first(), S1_vis, "corrected ascending")
Map.addLayer(corrected_asc.first(), S1_layover, "layover ascending")
Map.addLayer(corrected_asc.first(), S1_shadow, "shadow ascending")

Map.addLayer(s1_stack_desc.first(), S1_vis, "original descending")
Map.addLayer(corrected_desc.first(), S1_vis, "corrected descending")
Map.addLayer(corrected_desc.first(), S1_layover, "layover descending")
Map.addLayer(corrected_asc.first(), S1_shadow, "shadow descending")

Map.addLayerControl()
Map.centerObject(squamish,10)
Map
